In [658]:
import matplotlib.pyplot as plt
import seaborn as sns
# Configs
%config InlineBackend.figure_format ='retina'
%matplotlib inline

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn import externals, base, neural_network, neighbors, feature_extraction, datasets, linear_model, metrics, model_selection, preprocessing, svm, ensemble, decomposition, feature_extraction, utils

import keras


In [659]:
# Load Bridge helper classes
%run helpers/bridge.ipynb
%run helpers/load-data.ipynb
%run helpers/train-eval-utils.ipynb

In [660]:
# Loads & precomputes all the data, takes about 10min
data = load_deals()

# Helpers

In [667]:
class Strategy(object):
    
    def __init__(self, function):
        # Takes as an input a function from (Hand, Bidding) -> bid
        self.fn = function
        
    def step(self, hand, bidding):
        # In case the bid is invalid, automatically substitute for PASS (idk)
        bid = self.fn(hand, bidding)
        if isinstance(bid, str):
            bid = valid_bids.index(bid)
        if 0 < bid <= len(contracts) and bid > bidding.last_bid():
            return bidding.add(self.fn(hand, bidding))
        return bidding.add(0)
    
    def evaluate(self, list_of_deals, enemy_s=Strategy(pass_function), with_opps=True):
        if not isinstance(list_of_deals, list):
            list_of_deals = [list_of_deals]
        rt = 0
        for d in list_of_deals:
            seq = gen_sequence(d, self, enemy_s, self)
            res, side = seq.result()
            rt += d.ev(res, side, with_opps=with_opps)
        return rt / len(list_of_deals)

In [121]:
def gen_sequence(deal, s1, s2=Strategy(pass_function), s3=None, s4=None):
    seq = Bidding()
    if s3 is None:
        s3 = s1
    if s4 is None:
        s4 = s2
    sgs = [s1, s2, s3, s4]
    while not seq.done:
        cur_pl = len(seq) % 4
        seq = sgs[cur_pl].step(deal.hands[cur_pl], seq)
    return seq

# Examples

In [ ]:
# Full knowledge ~ Theoretical limit

In [ ]:
# TODO

## One hand places the contract

In [219]:
def const_function(contr):
    def fn(hand, bidding):
        return contr
    return fn

In [ ]:
def pass_function(hand, bidding):
    return const_function('PASS')

In [651]:
def hcp_function(start=14.2, step=3.9):
    def fn(hand, bidding):
        ft = hand.features()
        if ft['hcp'] >= start:
            lev = min(7, (ft['hcp'] - start) / step + 1)
            return str(int(lev)) + 'NT'
        return 'PASS'
    return fn

In [650]:
def suit_function(start=12.8, step=2.6):
    def fn(hand, bidding):
        ft = hand.features()
        if ft['hcp'] >= start:
            lev = min(7, (ft['hcp'] - start) / step + 1)
            strain = 'NT'
            bst_len = 4
            for s in suits:
                if ft[s + '_len'] >= bst_len:
                    strain = s
                    bst_len = ft[s + '_len']
            return str(int(lev)) + strain
            #return const_function(str(int(lev)) + strain)(hand, bidding)
        return 'PASS'
    return fn

In [552]:
def strategy_from_model(model):
    def fn(hand, bidding):
        bid = model.predict(hand.feature_array().reshape(1, -1))[0]
        return bid
    return fn

### Theoretical limit

In [ ]:
# Try to make a NN learn the best contract with_opps

In [560]:
X = np.array([d.hands[0].feature_array() for d in data[:2 * 10 ** 4]]) #only N hand

# Winners in a particular strain played from N
y = [(['PASS'] + contracts).index(d.best_contract(with_opps=True))
     for d in data[:2 * 10 ** 4]]

In [561]:
# This model overfits terribly

In [570]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(256, 256, 256, 256,), 
                                     max_iter=200, 
                                     #early_stopping=True, tol=1e-6
                                    )
model.fit(X, y) # 3min
print(model.n_iter_)

141


In [571]:
from sklearn.externals import joblib
joblib.dump(model, 'models/Nhands_to_contracts_with.pkl') 

['models/Nhands_to_contracts_with.pkl']

In [572]:
# Load the pickle file
model_load = joblib.load('models/Nhands_to_contracts_with.pkl') 
model.score(X, y)

0.96585

In [576]:
# Model overfits terribly
model.score(X_ts, y_ts)

0.0702

In [573]:
eval_strategy(data[:2 * 10 ** 4], Strategy(strategy_from_model(model_load)))

-5.7913

In [574]:
eval_strategy(data[2 * 10 ** 4:], Strategy(strategy_from_model(model_load)))

-10.552

## One hand describes, other places

In [644]:
def base_function(hand, bidding):
    bids_made = len(bidding)
    ft = hand.features()
    
    min_opening_str = 10
    hcp_step = 4
    card_fit_suit = 8
    
    # if other hand has not described
    if bids_made <= 1:
        if ft['hcp'] < min_opening_str:
            return 'PASS'
        else:
            # [10, 13] [14, 17] ...
            lev = min(7, (ft['hcp'] - min_opening_str) / hcp_step + 1)
            # Bids a 5 card suit if it has it, else NT
            strain = 'NT'
            bst_len = 5
            for s in suits:
                if ft[s + '_len'] >= bst_len:
                    strain = s
                    bst_len = ft[s + '_len']
            return str(int(lev)) + strain
        
    # if other hand has described
    if 1 < bids_made <= 3:
        p_bid = bidding.partner_bids()[0]
        # Estimates hcp
        if p_bid == 'PASS':
            ft['hcp'] += min_opening_str - 3
        else:
            ft['hcp'] += min_opening_str - 3 + hcp_step * int(p_bid[0])
        # Estimates distribution as 3333 or 5222
        if p_bid == 'PASS' or p_bid[1:] == 'NT':
            for s in suits:
                ft[s + '_len'] += 3
        else:
            for s in suits:
                if s == p_bid[1:]:
                    ft[s + '_len'] += 5
                else:
                    ft[s + '_len'] += 2
        # Guesses best contract
        # strain = s if >= 8 card fit, else NT
        strain = 'NT'
        bst_len = card_fit_suit
        for s in suits:
            if ft[s + '_len'] >= bst_len:
                strain = s
                bst_len = ft[s + '_len']
                
        # level for NT 19, 22, 25, 28, 31, 34, 37
        if strain == 'NT':
            level = (ft['hcp'] - 16) // 3
        # level for suit depends on length
        else:
            # hcp + 2*length >= 25 + 16 for game
            level = (ft['hcp'] + 2 * ft[strain + '_len'] - 29) // 3
        level = min(7, level)

        # Principle of suit correction:
        #  if p made a bid in a NT strain we might wish to raise the level
        #  to change that to a suit
        if p_bid[1:] == 'NT':
            if strain != 'NT' and level <= int(p_bid[0]):
                level = int(p_bid[0]) + 1
            
        # for minor suits, correct 4m and 5m to 3NT
        if strain in ['c', 'd'] and 4 <= level <= 5:
            level = 3; strain = 'NT'
        # correct non-slams above game
        if strain in ['h', 's'] and level == 5:
            level = 4
        if strain == 'NT' and 4 <= level <= 5:
            level = 3


        if level <= 0:
            return 'PASS'
        else:
            return str(level) + strain
    
    # if other hand has placed the contract
    if bids_made >= 4:
        return 'PASS'

In [645]:
myd = Deal()
print(myd.pbn())
gen_sequence(myd, Strategy(base_function))

T9.Q652.842.AKQ8 K53.KJT84.J3.532 Q8742.3.AK7.J976 AJ6.A97.QT965.T4


N: 1NT - (P) -  2s - (P) - 
   (P) - (P) - 

In [646]:
for idx, d in enumerate(data[:100]):
    if eval_strategy(data[idx:idx+1], Strategy(base_function)) < -5:
        print (d.pbn())
        print (eval_strategy(data[idx:idx+1], Strategy(base_function)))
        print (gen_sequence(d, Strategy(base_function)))
        print ("\n\n")

AKJ852.AKT3..KT9 T3.Q62.5.AQJ8763 Q764.98754.J872. 9.J.AKQT9643.542
-13.0
N:  3s - (P) - (P) - (P) - 
   



8.K7654.K852.KT8 KQ76.AT2.3.AJ542 AJT932.Q.AQ97.93 54.J983.JT64.Q76
-6.0
N: (P) - (P) -  3s - (P) - 
   (P) - (P) - 



J972.K2.Q963.A75 K853.Q874.A5.KJ2 AQ.AJT93.4.QT843 T64.65.KJT872.96
-8.0
N: 1NT - (P) -  3h - (P) - 
   (P) - (P) - 



J.KQ63.KJ82.K652 AK7.J9.A654.AJ74 T96542.54.T3.T98 Q83.AT872.Q97.Q3
-9.0
N: 1NT - (P) -  2s - (P) - 
   (P) - (P) - 



AKJ976..J876.863 543.86.T954.KJ95 QT8.QT953.AK3.72 2.AKJ742.Q2.AQT4
-13.0
N: (P) - (P) -  1h - (P) - 
   (P) - (P) - 



A.AQJ542.8.AQ643 962.K9763.AJ73.2 543.T8.KQ52.KJ97 KQJT87..T964.T85
-12.0
N:  2h - (P) - 2NT - (P) - 
   (P) - (P) - 



J842.T86.T92.J62 KQT3.KQJ95.843.3 9.A72.AKQJ.AQ987 A765.43.765.KT54
-12.0
N: (P) - (P) - 3NT - (P) - 
   (P) - (P) - 



A742.63.AQT.AQJ6 KQT5.QJ952.J6.K4 J98.K8.9732.8532 63.AT74.K854.T97
-9.0
N: 2NT - (P) - (P) - (P) - 
   



AT.AJ84.KJ65.QT2 J9762.QT72.3.J84 Q5.K9.AQT98.A763 K843.653.

In [670]:
Strategy(pass_function).evaluate(data)

-4.547764336291191

In [671]:
Strategy(hcp_function()).evaluate(data)

-4.334017286610689

In [ ]:
Strategy(suit_function()).evaluate(data)

In [ ]:
Strategy(base_function).evaluate(data)

In [ ]:
len(data)